# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [ ]:
# your code goes here

In [ ]:
import pandas as pd
url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv"
df = pd.read_csv(url)

In [1]:
# (Optional) Inspect the data to confirm column names
print(df.columns)

# Filter the DataFrame
filtered_df = df[(df["Total Claim Amount"] > 1000) & (df["Response"] == "Yes")]

# View the result
filtered_df.head()


Index(['Unnamed: 0', 'Customer', 'State', 'Customer Lifetime Value',
       'Response', 'Coverage', 'Education', 'Effective To Date',
       'EmploymentStatus', 'Gender', 'Income', 'Location Code',
       'Marital Status', 'Monthly Premium Auto', 'Months Since Last Claim',
       'Months Since Policy Inception', 'Number of Open Complaints',
       'Number of Policies', 'Policy Type', 'Policy', 'Renew Offer Type',
       'Sales Channel', 'Total Claim Amount', 'Vehicle Class', 'Vehicle Size',
       'Vehicle Type'],
      dtype='object')


,Unnamed: 0,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,...,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size,Vehicle Type
189,189,OK31456,California,11009.130490,Yes,Premium,Bachelor,1/24/11,Employed,F,...,0.0,1,Corporate Auto,Corporate L3,Offer2,Agent,1358.400000,Luxury Car,Medsize,NaN
236,236,YJ16163,Oregon,11009.130490,Yes,Premium,Bachelor,1/24/11,Employed,F,...,0.0,1,Special Auto,Special L3,Offer2,Agent,1358.400000,Luxury Car,Medsize,A
419,419,GW43195,Oregon,25807.063000,Yes,Extended,College,2/13/11,Employed,F,...,1.0,2,Personal Auto,Personal L2,Offer1,Branch,1027.200000,Luxury Car,Small,A
442,442,IP94270,Arizona,13736.132500,Yes,Premium,Master,2/13/11,Disabled,F,...,0.0,8,Personal Auto,Personal L2,Offer1,Web,1261.319869,SUV,Medsize,A
587,587,FJ28407,California,5619.689084,Yes,Premium,High School or Below,1/26/11,Unemployed,M,...,0.0,1,Personal Auto,Personal L1,Offer2,Web,1027.000029,SUV,Medsize,A


In [ ]:
Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" 
to the last marketing campaign. Write your conclusions.

In [3]:
# Load the original dataset
url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv"
df = pd.read_csv(url)

# Filter to include only those who responded "Yes" to the last marketing campaign
df_yes = df[df["Response"] == "Yes"]


In [5]:
# Calculate the average (mean) total claim amount by policy type and gender
summary_df = (
    df_yes
    .groupby(["Policy Type", "Gender"])["Total Claim Amount"]
    .mean()
    .reset_index()
)

summary_df.rename(columns={"Total Claim Amount": "Avg Total Claim Amount"}, inplace=True)
summary_df

print("  ")


,Policy Type,Gender,Avg Total Claim Amount
0,Corporate Auto,F,433.738499
1,Corporate Auto,M,408.582459
2,Personal Auto,F,452.965929
3,Personal Auto,M,457.010178
4,Special Auto,F,453.280164
5,Special Auto,M,429.527942


In [ ]:
Analyze the total number of customers who have policies in each state, and then filter the results to only include states 
where there are more than 500 customers

In [7]:
# Group by state and count the number of customers
counts_by_state = df.groupby('State')['Customer'].count().reset_index(name='Num_Customers')

# Filter to only include states where there are more than 500 customers
states_over_500 = counts_by_state[counts_by_state['Num_Customers'] > 500]

states_over_500


,State,Num_Customers
0,Arizona,1937
1,California,3552
2,Nevada,993
3,Oregon,2909
4,Washington,888


In [ ]:
Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [9]:
print(df.columns)

Index(['Unnamed: 0', 'Customer', 'State', 'Customer Lifetime Value',
       'Response', 'Coverage', 'Education', 'Effective To Date',
       'EmploymentStatus', 'Gender', 'Income', 'Location Code',
       'Marital Status', 'Monthly Premium Auto', 'Months Since Last Claim',
       'Months Since Policy Inception', 'Number of Open Complaints',
       'Number of Policies', 'Policy Type', 'Policy', 'Renew Offer Type',
       'Sales Channel', 'Total Claim Amount', 'Vehicle Class', 'Vehicle Size',
       'Vehicle Type'],
      dtype='object')


In [11]:
# Group by Education and Gender to find max, min, and median of CLV
stats_df = (
    df
    .groupby(["Education", "Gender"])["Customer Lifetime Value"]
    .agg(["max", "min", "median"])
    .reset_index()
)

stats_df


,Education,Gender,max,min,median
0,Bachelor,F,73225.95652,1904.000852,5640.505303
1,Bachelor,M,67907.27050,1898.007675,5548.031892
2,College,F,61850.18803,1898.683686,5623.611187
3,College,M,61134.68307,1918.119700,6005.847375
4,Doctor,F,44856.11397,2395.570000,5332.462694
5,Doctor,M,32677.34284,2267.604038,5577.669457
6,High School or Below,F,55277.44589,2144.921535,6039.553187
7,High School or Below,M,83325.38119,1940.981221,6286.731006
8,Master,F,51016.06704,2417.777032,5729.855012
9,Master,M,50568.25912,2272.307310,5579.099207


In [ ]:
BONUS

In [ ]:
The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged 
as columns and the states are arranged as rows.

In [21]:

import pandas as pd
url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv"
df = pd.read_csv(url)

#Convert "Effective To Date" to a datetime type and extract the month name
df["Effective To Date"] = pd.to_datetime(df["Effective To Date"])
df["Month"] = df["Effective To Date"].dt.month_name()

# Create a pivot table
#    Index = State, Columns = Month, Values = any column that uniquely identifies a policy (e.g. Customer)
#    We use aggfunc='count' to count the number of policies
policies_by_state_month = pd.pivot_table(
    df,
    index="State",
    columns="Month",
    values="Customer",
    aggfunc="count",
    fill_value=0  # Fill missing combinations with 0
)

C:\Users\graci\AppData\Local\Temp\ipykernel_32520\2767932877.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Effective To Date"] = pd.to_datetime(df["Effective To Date"])


In [23]:
# (Optional) Ensure months are in chronological order, if you prefer
month_order = ["January","February","March","April","May","June","July","August","September","October","November","December"]
policies_by_state_month = policies_by_state_month.reindex(columns=month_order)

#Display the table
policies_by_state_month


Month,January,February,March,April,May,June,July,August,September,October,November,December
State,,,,,,,,,,,,
Arizona,1008,929,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
California,1918,1634,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Nevada,551,442,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Oregon,1565,1344,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Washington,463,425,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
Display a new DataFrame that contains the number of policies sold by month, by state, 
for the top 3 states with the highest number of policies sold.

In [25]:

# (Optional) Convert "Effective To Date" to datetime.
# Adjust 'format' and 'errors' as needed based on actual date formats.
df["Effective To Date"] = pd.to_datetime(df["Effective To Date"], errors='coerce')

# Extract the month name
df["Month"] = df["Effective To Date"].dt.month_name()

#Group by state and month, counting the number of policies (assuming each row = 1 policy)
grouped = df.groupby(["State", "Month"])["Customer"].count().reset_index(name="Num_Policies")

# Determine the top 3 states by total policies sold
top_states = (
    grouped
    .groupby("State")["Num_Policies"]
    .sum()
    .reset_index()
    .sort_values("Num_Policies", ascending=False)
)

# Select only the top 3 states
top_3 = top_states["State"].head(3).tolist()

# Filter data to include only those top 3 states
filtered = grouped[grouped["State"].isin(top_3)]

# Pivot the data so months are rows, states are columns, and the cell values are policies sold
result = filtered.pivot_table(
    index="Month",
    columns="State",
    values="Num_Policies",
    aggfunc="sum",
    fill_value=0
)

# (Optional) Reorder months from January to December
months_order = [
    "January","February","March","April","May","June",
    "July","August","September","October","November","December"
]
result = result.reindex(index=months_order)

# Display the final DataFrame
result


State,Arizona,California,Oregon
Month,,,
January,1008.0,1918.0,1565.0
February,929.0,1634.0,1344.0
March,NaN,NaN,NaN
April,NaN,NaN,NaN
May,NaN,NaN,NaN
June,NaN,NaN,NaN
July,NaN,NaN,NaN
August,NaN,NaN,NaN
September,NaN,NaN,NaN


In [ ]:
The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

In [27]:
import pandas as pd

url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv"
df = pd.read_csv(url)


In [28]:
# Create a contingency table: rows = sales channels, columns = responses
response_table = pd.crosstab(index=df["Sales Channel"], columns=df["Response"])

response_table


Response,No,Yes
Sales Channel,,
Agent,3148,742
Branch,2539,326
Call Center,1792,221
Web,1334,177


In [31]:
# Sum across columns to get total responses by channel
response_table["Total"] = response_table["No"] + response_table["Yes"]

# Calculate the proportion (or percentage) of "Yes" responses
response_table["Yes_Rate"] = response_table["Yes"] / response_table["Total"]

response_table


Response,No,Yes,Total,Yes_Rate
Sales Channel,,,,
Agent,3148,742,3890,0.190746
Branch,2539,326,2865,0.113787
Call Center,1792,221,2013,0.109786
Web,1334,177,1511,0.117141


In [ ]:
**exemple with the HINT**

In [33]:
# Reset index so "Sales Channel" becomes a column
response_melted = response_table.reset_index().melt(
    id_vars=["Sales Channel"], 
    value_vars=["No", "Yes", "Total", "Yes_Rate"],
    var_name="Metric",
    value_name="Value"
)

response_melted


,Sales Channel,Metric,Value
0,Agent,No,3148.000000
1,Branch,No,2539.000000
2,Call Center,No,1792.000000
3,Web,No,1334.000000
4,Agent,Yes,742.000000
5,Branch,Yes,326.000000
6,Call Center,Yes,221.000000
7,Web,Yes,177.000000
8,Agent,Total,3890.000000
9,Branch,Total,2865.000000
